In [2]:
!pip install pymupdf pandas streamlit openai





   ---------------------------------------- 0.0/18.5 MB ? eta -:--:--
   -- ------------------------------------- 1.3/18.5 MB 8.4 MB/s eta 0:00:03
   --------- ------------------------------ 4.5/18.5 MB 10.7 MB/s eta 0:00:02
   -------------- ------------------------- 6.6/18.5 MB 10.9 MB/s eta 0:00:02
   ------------------- -------------------- 8.9/18.5 MB 11.1 MB/s eta 0:00:01
   -------------------------- ------------- 12.1/18.5 MB 11.6 MB/s eta 0:00:01
   ------------------------------- -------- 14.7/18.5 MB 12.0 MB/s eta 0:00:01
   -------------------------------------- - 17.8/18.5 MB 12.2 MB/s eta 0:00:01
   ---------------------------------------- 18.5/18.5 MB 11.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/730.3 kB ? eta -:--:--
   --------------------------------------- 730.3/730.3 kB 14.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 18.0 MB/s eta 0:00:00
  Atte


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import fitz  # PyMuPDF

def extract_text_from_pdf(file_path):
    """Extracts all text from a PDF file."""
    with fitz.open(file_path) as doc:
        return "\n".join(page.get_text() for page in doc)

# Provide full path to your PDF file here
pdf_file = r"C:\Users\LENOVO\Downloads\sampledata.doc.pdf"

# function call
pdf_text = extract_text_from_pdf(pdf_file)

# Print first 1200 characters for preview
print(pdf_text[:1200])


                       UPI Transaction Statement 
------------------------------------------------------------ 
 
Name           : Mr.Wilson 
Account Number : 1234 5678 9012 
UPI ID         : ramesh.kumar@okicici 
Address        : D.No: 12-34-56, Sai Nagar, Vizag 
City           : Visakhapatnam, Andhra Pradesh 
PIN            : 530012 
Bank Name      : ICICI Bank 
 
Statement Period: 01/04/2025 to 30/06/2025 
 
 
Date         | Time   | Amount    | Merchant         | Description 
--------------------------------------------------------------- 
01/04/2025   | 10:15  | -350.00   | Zomato           | Food Delivery 
03/04/2025   | 15:45  | -1200.00  | Flipkart         | Shopping 
05/04/2025   | 12:10  | +15000.00 | XYZ Pvt Ltd      | Salary Credit 
10/04/2025   | 18:30  | -500.00   | Amazon           | Shopping 
15/04/2025   | 09:50  | -75.00    | Swiggy           | Snacks 
22/04/2025   | 19:00  | -2000.00  | BookMyShow       | Movie Tickets 
28/04/2025   | 11:20  | -850.00   | Uber       

In [7]:
import fitz  # PyMuPDF for reading PDF files
import pandas as pd  # for handling tabular data (rows & columns)
import re  # for regular expressions (pattern matching)

# 1️ Function to Extract All Text from PDF
def extract_text_from_pdf(file_path):
    """
    Extracts all text from a PDF file.
    """
    with fitz.open(file_path) as doc:
        return "\n".join(page.get_text() for page in doc)

# 2️ Function to Extract Transactions and Structure Them into Table (DataFrame)
def extract_transactions_to_dataframe(text):
    """
    Processes the raw PDF text and extracts transaction data into a structured DataFrame.
    """
    lines = text.split('\n')  # split text line by line
    data = []

    # Regular expression pattern to capture Date, Time, Amount, Receiver, Description
    pattern = re.compile(r"(\d{2}/\d{2}/\d{4})\s*\|\s*(\d{2}:\d{2})\s*\|\s*([+-]?\d+\.\d{2})\s*\|\s*(.*?)\s{2,}\|\s*(.*)")

    # Mapping merchants to categories (to automatically assign categories)
    category_keywords = {
        'Food': ['Zomato', 'Swiggy', 'Uber Eats', 'Starbucks', 'Domino', 'Breakfast'],
        'Shopping': ['Amazon', 'Flipkart', 'Myntra', 'Big Bazaar', 'Reliance'],
        'Salary': ['Salary', 'Project Payment', 'Freelance'],
        'Entertainment': ['BookMyShow'],
        'Recharge': ['Tata Sky'],
        'Transport': ['Uber']
    }

    # Helper function to assign category based on merchant or description
    def get_category(merchant, description):
        for category, keywords in category_keywords.items():
            if any(k.lower() in (merchant + description).lower() for k in keywords):
                return category
        return 'Other'

    # Loop through each line of text
    for line in lines:
        match = pattern.match(line.strip())  # check if line matches the pattern
        if match:
            date, time, amount, merchant, description = match.groups()
            category = get_category(merchant, description)  # get category for each transaction
            data.append({
                'Date': date,
                'Time': time,
                'Amount': float(amount),
                'Receiver': merchant.strip(),
                'Description': description.strip(),
                'Category': category
            })

    return pd.DataFrame(data)

# 3️ Path to Your PDF File (Change this path to match your system)
pdf_path = r"C:\Users\LENOVO\Downloads\sampledata.doc.pdf"

# 4️ Extract Text from PDF
pdf_text = extract_text_from_pdf(pdf_path)

# 5️ Process Text to Get Structured Transactions
df = extract_transactions_to_dataframe(pdf_text)

# 6️ Save Output to CSV & JSON
df.to_csv("upi_transactions.csv", index=False)
df.to_json("upi_transactions.json", orient='records', indent=4)

# 7️ Preview the Data in Terminal
print(df)


          Date   Time   Amount         Receiver        Description  \
0   01/04/2025  10:15   -350.0           Zomato      Food Delivery   
1   03/04/2025  15:45  -1200.0         Flipkart           Shopping   
2   05/04/2025  12:10  15000.0      XYZ Pvt Ltd      Salary Credit   
3   10/04/2025  18:30   -500.0           Amazon           Shopping   
4   15/04/2025  09:50    -75.0           Swiggy             Snacks   
5   22/04/2025  19:00  -2000.0       BookMyShow      Movie Tickets   
6   28/04/2025  11:20   -850.0             Uber           Cab Ride   
7   02/05/2025  09:00   5000.0      ABC Pvt Ltd  Freelance Payment   
8   06/05/2025  20:15   -450.0  Reliance Trends           Clothing   
9   11/05/2025  14:40   -900.0           Amazon        Electronics   
10  17/05/2025  16:25   -350.0         Domino’s              Pizza   
11  20/05/2025  13:35   -200.0        Starbucks             Coffee   
12  25/05/2025  10:45  -1200.0         Tata Sky           Recharge   
13  31/05/2025  08:3

In [9]:
# Function to categorize transactions based on keywords in RECEIVER (not description)
def categorize(receiver):
    # Convert to lowercase for matching
    rec = receiver.lower()

    #  categories and keywords
    categories = {
        'Food': ['zomato', 'swiggy', 'uber eats', 'starbucks', 'domino', 'breakfast'],
        'Shopping': ['amazon', 'flipkart', 'myntra', 'big bazaar', 'reliance'],
        'Salary': ['salary', 'project payment', 'freelance'],
        'Entertainment': ['bookmyshow'],
        'Recharge': ['tata sky'],
        'Transport': ['uber']
    }

    # Check each category for match
    for category, keywords in categories.items():
        if any(keyword in rec for keyword in keywords):
            return category
    return 'Others'


# Apply the categorize() function on the **Receiver** column
df['Category'] = df['Receiver'].apply(categorize)

# Save as CSV
df.to_csv("categorized_transactions.csv", index=False)

# Preview result
print(df[['Date', 'Receiver', 'Description', 'Category']].head())


         Date     Receiver    Description  Category
0  01/04/2025       Zomato  Food Delivery      Food
1  03/04/2025     Flipkart       Shopping  Shopping
2  05/04/2025  XYZ Pvt Ltd  Salary Credit    Others
3  10/04/2025       Amazon       Shopping  Shopping
4  15/04/2025       Swiggy         Snacks      Food


In [10]:
#  Convert the 'Date' column to datetime objects (dayfirst=True because format is DD/MM/YYYY)
df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)

#  Create a new column 'Month' to store Month-Year format (e.g., Apr-2025)
df["Month"] = df["Date"].dt.strftime("%b-%Y")

#  Group the transactions by 'Month' and calculate total spending (sum of Amounts per month)
monthly_spending = df.groupby("Month")["Amount"].sum()

#  Display the monthly spending summary
print("📆 Monthly Spending:\n", monthly_spending)


📆 Monthly Spending:
 Month
Apr-2025    10025.0
Jul-2025    -5950.0
Jun-2025     2050.0
May-2025     1625.0
Name: Amount, dtype: float64


In [11]:
import pandas as pd

#  Load your structured CSV file (already extracted from PDF and categorized)
df = pd.read_csv("categorized_transactions.csv")

#  Convert 'Date' to datetime format to extract Month easily
df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)

#  Create a 'Month' column in "MMM-YYYY" format (e.g., Apr-2025)
df["Month"] = df["Date"].dt.strftime("%b-%Y")

#  Group by both 'Month' and 'Category', then calculate total spending
monthly_category_spending = df.groupby(["Month", "Category"])["Amount"].sum().reset_index()

# Show result neatly
print(monthly_category_spending)

#  Save to CSV .
monthly_category_spending.to_csv("monthly_category_spending.csv", index=False)


       Month       Category   Amount
0   Apr-2025  Entertainment  -2000.0
1   Apr-2025           Food   -425.0
2   Apr-2025         Others  15000.0
3   Apr-2025       Shopping  -1700.0
4   Apr-2025      Transport   -850.0
5   Jul-2025  Entertainment   -250.0
6   Jul-2025           Food   -700.0
7   Jul-2025       Shopping  -5000.0
8   Jun-2025  Entertainment   -250.0
9   Jun-2025           Food   -700.0
10  Jun-2025         Others   8000.0
11  Jun-2025       Shopping  -5000.0
12  May-2025           Food   -825.0
13  May-2025         Others   5000.0
14  May-2025       Recharge  -1200.0
15  May-2025       Shopping  -1350.0


columns add to show month details also full transaction 

In [15]:
import pandas as pd

#  Load your structured CSV file ( Date, Amount, Description)
df = pd.read_csv("categorized_transactions.csv")

#  Ensure 'Date' is in datetime format
df["Date"] = pd.to_datetime(df["Date"], dayfirst=True, errors='coerce')
df.dropna(subset=["Date"], inplace=True)

#  Create 'Month' column in "MMM-YYYY" format
df["Month"] = df["Date"].dt.strftime("%b-%Y")

#  Rearrange or  columns
df = df[["Date", "Month", "Description", "Category", "Amount"]]

#  Save to CSV including Date, Month, Description, Category, Amount
df.to_csv("full_transactions_with_month.csv", index=False)

#   Group by 'Month' + 'Category' → total spending
monthly_category_spending = df.groupby(["Month", "Category"])["Amount"].sum().reset_index()

#  Save summarized version too (optional)
monthly_category_spending.to_csv("monthly_category_spending2.csv", index=False)

#  Print to confirm
print(" Full transactions with Date, Month, Description, Category, Amount:")
print(df.head())

print("\n Monthly + Category total spending summary:")
print(monthly_category_spending)


 Full transactions with Date, Month, Description, Category, Amount:
        Date     Month    Description  Category   Amount
0 2025-04-01  Apr-2025  Food Delivery      Food   -350.0
1 2025-04-03  Apr-2025       Shopping  Shopping  -1200.0
2 2025-04-05  Apr-2025  Salary Credit    Others  15000.0
3 2025-04-10  Apr-2025       Shopping  Shopping   -500.0
4 2025-04-15  Apr-2025         Snacks      Food    -75.0

 Monthly + Category total spending summary:
       Month       Category   Amount
0   Apr-2025  Entertainment  -2000.0
1   Apr-2025           Food   -425.0
2   Apr-2025         Others  15000.0
3   Apr-2025       Shopping  -1700.0
4   Apr-2025      Transport   -850.0
5   Jul-2025  Entertainment   -250.0
6   Jul-2025           Food   -700.0
7   Jul-2025       Shopping  -5000.0
8   Jun-2025  Entertainment   -250.0
9   Jun-2025           Food   -700.0
10  Jun-2025         Others   8000.0
11  Jun-2025       Shopping  -5000.0
12  May-2025           Food   -825.0
13  May-2025         Others

In [15]:
! pip install openai


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Import necessary libraries
import pandas as pd
import openai  # Requires 'openai' 



# -------------------
# Step 2: Preprocess - Convert Date to datetime and extract Month
df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)
df["Month"] = df["Date"].dt.strftime("%b-%Y")

# -------------------
# Step 3: Summarize Spending
monthly_spending = df.groupby("Month")["Amount"].sum()
category_spending = df.groupby("Category")["Amount"].sum()

# -------------------
# Step 4: Format Data for LLM Prompt
monthly_spending_str = "\n".join(
    [f"{month}    ₹{amount:,.2f}" for month, amount in monthly_spending.items()]
)

category_spending_str = "\n".join(
    [f"{category}: ₹{amount:,.2f}" for category, amount in category_spending.items()]
)

prompt = f"""
You are a financial advisor. Based on the following monthly and category-wise spending in Indian Rupees (₹), provide 3 personalized financial tips.

Monthly Spend:
{monthly_spending_str}

Category-wise Spend:
{category_spending_str}
"""

# -------------------
# Step 5: Call OpenRouter API (Langflow can use this part too)
client = openai.OpenAI(
    api_key="sk-or-api_key=\"your api key update ",  # OpenRouter API key
    base_url="https://openrouter.ai/api/v1"
)

response = client.chat.completions.create(
    model="openrouter/auto",  # Default reliable model
    messages=[
        {"role": "user", "content": prompt}
    ]
)

# -------------------
# Step 6: Display the Financial Advice
print("💡 Financial Advice:\n")
print(response.choices[0].message.content)


💡 Financial Advice:

**Personalized Financial Tips Based on Your Spending Patterns:**

1. **Control Discretionary Spending (Especially Shopping):**  
   Your "Shopping" category (₹13,050) accounts for **~65% of total expenses**, which is disproportionately high. This could indicate impulsive purchases or lifestyle inflation.  
   **Action:** Set a strict monthly shopping budget (e.g., 15-20% of total expenses) and use a 24-hour "cooling-off" period before making non-essential purchases. Track smaller shopping expenses (e.g., online orders) that may add up quickly.

2. **Clarify Income/Expense Categorization:**  
   The "Others" category shows a *positive* ₹28,000, which might be mislabeled income. Meanwhile, monthly spending data (e.g., ₹10,025 in April vs. ₹-5,950 in July) suggests irregular cash flow or possible accounting errors.  
   **Action:**  
   - Separate *income* and *expenses* clearly (e.g., label "Salary" or "Side Income" for ₹28,000).  
   - Use budgeting apps to track tr

In [14]:
import pandas as pd

df = pd.read_csv(r'C:\Users\LENOVO\Desktop\Newfolder\project6\upi_transactions.csv')
print(df.head())


         Date   Time   Amount     Receiver    Description  Category
0  01/04/2025  10:15   -350.0       Zomato  Food Delivery      Food
1  03/04/2025  15:45  -1200.0     Flipkart       Shopping  Shopping
2  05/04/2025  12:10  15000.0  XYZ Pvt Ltd  Salary Credit    Salary
3  10/04/2025  18:30   -500.0       Amazon       Shopping  Shopping
4  15/04/2025  09:50    -75.0       Swiggy         Snacks      Food
